In [15]:
import numpy as np

from qiskit import pulse
from qiskit.test.mock import FakeArmonk
from qiskit.providers.aer import PulseSimulator
from qiskit.providers.aer.pulse import PulseSystemModel
from qiskit.compiler import assemble
from qiskit.pulse import DriveChannel

armonk_backend = FakeArmonk()
armonk_model = PulseSystemModel.from_backend(armonk_backend)
channel = DriveChannel(0)
backend_sim = PulseSimulator()

num_shots = 1024

def objective_function(amplitudes):
    with pulse.build(armonk_backend) as x_sched:
#          pulse.play([(2 * np.random.rand() - 1) for i in range(100)], channel)
        new_amplitudes = []
        for x in amplitudes:
            if x > 1:
                x = 1
            if x < -1:
                x = -1
            new_amplitudes.append(x)
        pulse.play(new_amplitudes, channel)

    defaults = armonk_backend.defaults()
    config = armonk_backend.configuration()
    config.parametric_pulses=[]
    inst_sched_map = defaults.instruction_schedule_map
    measure = inst_sched_map.get('measure', qubits=config.meas_map[0])    
    x_sched += measure << x_sched.duration
    
    x_qobj = assemble(x_sched,
                  backend=backend_sim,
                  meas_level=2,
                  meas_return='single',
                  shots=num_shots)
    
    sim_result = backend_sim.run(x_qobj, armonk_model).result()
    counts = sim_result.get_counts()
    num_zeros = counts['0'] if '0' in counts else 0
    num_ones = counts['1'] if '1' in counts else 0 
    exp_val = (num_zeros - num_ones) / num_shots
    
    return exp_val

In [16]:
objective_function([(2 * np.random.rand() - 1) for i in range(100)])

<ipython-input-15-9f7dfa404b69>:42: DeprecationWarning: Passing `system_model` as a positional argument to `PulseSimulator.run` has been deprecated as of qiskit-aer 0.7.0 and will be removed no earlier than 3 months from that release date. Pass `system_model` as a kwarg `system_model=model` instead.
  sim_result = backend_sim.run(x_qobj, armonk_model).result()


0.974609375

In [19]:
from qiskit.aqua.components.optimizers import COBYLA

# Initialize the COBYLA optimizer
optimizer = COBYLA(maxiter=500, tol=0.0001)

# Create the initial parameters (noting that our single qubit variational form has 3 parameters)
params = np.random.rand(50)
ret = optimizer.optimize(num_vars=50, objective_function=objective_function, initial_point=params)

print(ret)

# # Obtain the output distribution using the final parameters
# qc = get_var_form(ret[0])
# t_qc = transpile(qc, backend)
# qobj = assemble(t_qc, shots=NUM_SHOTS)
# counts = backend.run(qobj).result().get_counts(qc)
# output_distr = get_probability_distribution(counts)

# print("Target Distribution:", target_distr)
# print("Obtained Distribution:", output_distr)
# print("Output Error (Manhattan Distance):", ret[1])
# print("Parameters Found:", ret[0])

<ipython-input-15-9f7dfa404b69>:42: DeprecationWarning: Passing `system_model` as a positional argument to `PulseSimulator.run` has been deprecated as of qiskit-aer 0.7.0 and will be removed no earlier than 3 months from that release date. Pass `system_model` as a kwarg `system_model=model` instead.
  sim_result = backend_sim.run(x_qobj, armonk_model).result()


(array([1.34923309, 1.69069282, 0.79893278, 1.83111558, 0.05015779,
       0.18822712, 1.84610588, 1.18234923, 1.25799425, 1.46921837,
       0.33898126, 0.86461043, 0.18700815, 1.86644592, 0.59660614,
       1.43991822, 0.53326173, 0.61902726, 0.99251688, 0.80971138,
       0.44728151, 0.64962727, 0.94120079, 0.96686075, 0.2597324 ,
       0.75577932, 0.93231129, 0.38628687, 0.17772601, 0.23907421,
       0.68047798, 0.71858744, 0.29223335, 0.06332095, 0.77519502,
       0.60432308, 0.13063246, 0.70301744, 0.48882472, 0.8558778 ,
       0.66104966, 0.22762655, 0.56980278, 0.50158693, 0.1817813 ,
       0.62211066, 0.97097953, 0.08690915, 0.5667201 , 0.02830087]), 0.732421875, 384)


In [58]:
backend_sim = PulseSimulator()

x_qobj = assemble(x_sched,
                  backend=backend_sim,
                  meas_level=2,
                  meas_return='single',
                  shots=512)

In [59]:
sim_result = backend_sim.run(x_qobj, armonk_model).result()
print(sim_result.get_counts())

{'0': 505, '1': 7}


<ipython-input-59-b8842ee6b3d0>:1: DeprecationWarning: Passing `system_model` as a positional argument to `PulseSimulator.run` has been deprecated as of qiskit-aer 0.7.0 and will be removed no earlier than 3 months from that release date. Pass `system_model` as a kwarg `system_model=model` instead.
  sim_result = backend_sim.run(x_qobj, armonk_model).result()
C:\Users\rdras\anaconda3\envs\plvqe_venv\lib\site-packages\qiskit\providers\aer\pulse\controllers\pulse_controller.py:142: UserWarning: Warning: qubit_lo_freq was not specified in PulseQobj and there is no default, so it is beign automatically determined from the drift Hamiltonian.
  warn('Warning: qubit_lo_freq was not specified in PulseQobj and there is no default, '


In [ ]:
defaults = backend.defaults()
config = backend.configuration()
config.parametric_pulses=[]
inst_sched_map = defaults.instruction_schedule_map
measure = inst_sched_map.get('measure', qubits=config.meas_map[0])
